# Forecast

In [1]:
from datetime import datetime, timedelta
from sqlalchemy import create_engine

import pandas as pd
import psycopg2

## Generate samples

In [4]:
def generate_sample(output):
    sql = '''
    INSERT INTO "Municipio".forecast 
    VALUES (%s,%s,'%s','%s',%s,\'%s\',\'%s\');
    '''.replace('\n', '').strip()
    
    CID10 = {
        'dengue': 'A90',
        'zika': 'A928',
        'chikungunya': 'A920'
    }
    
    cities = {
        'curitiba': 4106902,
        'rio': 3304557
    }
    
    for city_name, geoid in cities.items():
        for cid_name, cid_id in CID10.items():
            for w in range(1, 54):
                d = datetime.strptime('2017-01-01', '%Y-%m-%d') + timedelta(days=7*w)

                init_date = d.strftime('%Y-%m-%d')
                epiweek = '2017%s' % str(w).rjust(2, '0')
                cases = (w ** 2) * 10
                model_id = str(w)[0] * 128
                commit_id = str(w)[0] * 7

                text = sql % (
                    epiweek, geoid, cid_id,
                    init_date, cases, model_id, commit_id
                )

                print(text, file=output)

In [5]:
with open('/tmp/insert.sql', 'w') as f:
    generate_sample(f)

In [6]:
def get_passwd_from_ini(filepath: str):
    with open(filepath) as f:
        text = f.read()
        i = text.index('PSQL_PASSWORD')
        f = text[i:].index('\n')
        passwd = text[i:i+f].split('=')
        return passwd[1].strip()

In [7]:
db = 'dengue'
user = 'dengueadmin'
host = 'localhost'
passwd = get_passwd_from_ini('../AlertaDengue/AlertaDengue/settings.ini')

In [8]:
engine = create_engine("postgresql://{}:{}@{}/{}".format(
    user, passwd, host, db
))

In [9]:
sql = """
SELECT 
    (CASE 
     WHEN tb_cases."data_iniSE" IS NOT NULL 
       THEN tb_cases."data_iniSE" 
     ELSE forecast.init_date_epiweek
     END
    ) AS "data_iniSE",
    tb_cases.casos_est_min,
    tb_cases.casos_est,
    tb_cases.casos_est_max,
    tb_cases.casos,
    tb_cases.nivel,
    (CASE 
     WHEN tb_cases."SE" IS NOT NULL THEN tb_cases."SE" 
     ELSE forecast.epiweek
     END
    ) AS "SE",
    tb_cases.p_rt1,
    forecast.cases AS forecast_cases 
FROM
    (SELECT 
       data AS "data_iniSE",
       SUM(casos_estmin) AS casos_est_min,
       SUM(casos_est) as casos_est,
       SUM(casos_estmax) AS casos_est_max,
       SUM(casos) AS casos,
       MAX(nivel) AS nivel,
       se AS "SE",
       SUM(prt1) AS p_rt1
     FROM "Municipio".alerta_mrj
     GROUP BY "data_iniSE", "SE"
    ) AS tb_cases FULL OUTER JOIN "Municipio".forecast
      ON (tb_cases."SE" = forecast.epiweek)
ORDER BY "data_iniSE" ASC
"""

df = pd.read_sql(sql, con=engine)

In [10]:
df['casos_est'].isnull().any()

True